# DEDS WC 3.2

## Imports

In [46]:
import pyodbc
import sqlite3 as sql
import pandas as pd

## Connectie SSMS

In [47]:
DB_TOTAL = {
    "servername" : r"VISSIE\SQLEXPRESS",
    "database" : "SDM"}

export_conn_sdm = pyodbc.connect(f"""DRIVER={'SQL SERVER'};
                             SERVER={DB_TOTAL['servername']};
                             DATABASE={DB_TOTAL['database']};
                             Trusted_Connection=yes
                             """)

export_conn_sdm.setencoding('utf-8')
export_conn_sdm.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
export_conn_sdm.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')

export_cursor_sdm = export_conn_sdm.cursor()

DB_GO = {
    "servername" : r"VISSIE\SQLEXPRESS",
    "database" : "GREAT_OUTDOORS"}

export_conn_go = pyodbc.connect(f"""DRIVER={'SQL SERVER'};
                             SERVER={DB_GO['servername']};
                             DATABASE={DB_GO['database']};
                             Trusted_Connection=yes
                             """)

export_conn_go.setencoding('utf-8')
export_conn_go.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
export_conn_go.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')

export_cursor_go = export_conn_go.cursor()


## Dataframes

In [48]:
def create_dataframes_sql(connection):
    dictionary : dict = {}
    query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    key = "TABLE_NAME"
    
    tables = pd.read_sql(query, connection)
    
    for table in tables[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
    
    return dictionary

sdm_frames = create_dataframes_sql(export_conn_sdm)

C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_1476\973834393.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(query, connection)
C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_1476\973834393.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)


## Querie makers

In [49]:
from typing import Iterable

def query_remove(table_name : str):
    try:
        query = f"DELETE FROM {table_name}"
        export_cursor_go.execute(query)
        export_cursor_go.commit()
    except pyodbc.Error as e:
        print(f"ERROR: {table_name}: {e}")
        return table_name
    
    return None

def rigth_type(value, column_name, types):
    dtype = types[column_name]
    
    if pd.isna(value):
        return "NULL"
    
    if dtype == "object" or dtype == "string":
        value = value.replace("'", "''")
        return f"'{value}'"
    
    return f"{value}"
    

def create_add_query(row, types):
    query = ""
    columns = list(row.keys())
    
    for pos in range(len(columns)):
        column_name = columns[pos]
        value = row[column_name]
        
        if (pos == len(columns) - 1):
            query += f"{column_name}) VALUES ("
        else:
            query += f"{column_name}, "
    
    data = list(row)
    
    for pos in range(len(data)):
        column_name = columns[pos]
        value = row[column_name]
        
        if pos == len(columns) - 1:
            query += f"{rigth_type(value, column_name, types)})"
        else:
            query += f"{rigth_type(value, column_name, types)}, "
    
    return query

def query_add(table_name : str, table_data : pd.DataFrame):
    queries = []
    types = table_data.dtypes

    for index, row in table_data.iterrows():
        query : str = f"INSERT INTO {table_name} ("
        query += create_add_query(row, types)
        queries.append(query)
    
    return queries

## Leegmaken Great Outdoors database

In [50]:
def empty_go():
    tables_go : list = [
        "returned_item",
        "order_details",
        "retailer_site",
        "sales_staff",
        "inventory",
        "product",
        "product",
        "date"
    ]
    
    while len(tables_go) > 0:
        for table in tables_go:
            try:
                temp = query_remove(table)
                
                if (temp == None):
                    tables_go.remove(table)
                    print(f"REMOVED {table}")
                else:
                    print(f"NOT YET REMOVED: {table}")
            except pyodbc.Error as e:
                print(e)
        
    print("All items are removed")

empty_go()

REMOVED returned_item
REMOVED retailer_site
REMOVED inventory
REMOVED product
REMOVED order_details
REMOVED product
REMOVED sales_staff
REMOVED date
All items are removed


## Great Outdoors vullen

### Frames aanmaken

In [51]:
product_forecast_go = sdm_frames["forecast"]
product_forecast_go


,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME,FORECAST_ID
0,45,2023,12,90,1
1,46,2023,12,84,2
2,47,2023,12,468,3
3,48,2023,12,91,4
4,49,2023,12,35,5
...,...,...,...,...,...
3524,37,2022,1,60,3525
3525,38,2022,1,51,3526
3526,39,2022,1,235,3527
3527,40,2022,1,568,3528
